# Clase 5


Pasar lista

Contenidos:

* Uso avanzado de cobrapy: Construcción de redes metabólicas
* Análisis de soluciones alternativas de FBA`

# Uso avanzado de cobrapy: Construcción de una red metabólica


Ahora construiremos nuestra propia red metabólica usando como ejemplo la red metanogénica que vimos en una clase anterior:

<img src="https://raw.githubusercontent.com/modcommet/Clases/master/metanogenesis.png" width="400">


In [44]:
from cobra import Model, Reaction, Metabolite
cobra_model = Model('primordialMetabolism')

# Ractions
E1 = Reaction('E1')
E1.name = "Exchange reaction of CO2"
E1.subsystem = 'Entrance'
E1.lower_bound = 0.  
E1.upper_bound = 10.  

E2 = Reaction('E2')
E2.name = 'Exchange reaction of H2'
E2.subsystem = 'Entrance'
E2.lower_bound = 0.  
E2.upper_bound = 10.  

R1 = Reaction('R1')
R1.name = 'First reaction'
R1.subsystem = 'Internal reaction'
R1.lower_bound = 0.  
R1.upper_bound = 1000. 

R2 = Reaction('R2')
R2.name = 'Second reaction'
R2.subsystem = 'Internal reaction'
R2.lower_bound = 0.  
R2.upper_bound = 1000. 

R3 = Reaction('R3')
R3.name = 'Third reaction'
R3.subsystem = 'Internal reaction'
R3.lower_bound = 0.  
R3.upper_bound = 1000. 

E3 = Reaction('E3')
E3.name = 'Exchange reaction of F420'
E3.subsystem = 'Exit'
E3.lower_bound = 0.  
E3.upper_bound = 1000. 

E4 = Reaction('E4')
E4.name = 'Exchange reaction of CH4'
E4.subsystem = 'Exit'
E4.lower_bound = 0.  
E4.upper_bound = 1000.

E5 = Reaction('E5')
E5.name = 'Exchange reaction of H2O'
E5.subsystem = 'Exit'
E5.lower_bound = 0.  
E5.upper_bound = 1000.

# Metabolites
CO2 = Metabolite('CO2', formula='CO2',name='carbon dioxide',compartment='c')
H2 = Metabolite('H2', formula='H2',name='hydrogen',compartment='c')
F420 = Metabolite('F420',name='energy molecule',compartment='c')
CH4 = Metabolite('CH4',formula="CH4",name='methane',compartment='c')
I1 = Metabolite('I1',name='internal metabolite',compartment='c')
H2O = Metabolite('H2O',formula="H2O",name='water',compartment='c')

# Agrega metabolitos a las reacciones
E1.add_metabolites({CO2: 1.0})
E2.add_metabolites({H2: 2.0})
E3.add_metabolites({CH4: -1.0})
E4.add_metabolites({F420: -2})
E5.add_metabolites({H2O: -2})
R1.add_metabolites({F420: -1,CO2:-1,H2:-2,I1:1})
R2.add_metabolites({I1: -1,H2:-2,CH4:1,F420:1})
R3.add_metabolites({CH4: -1,I1:1,H2O:2,F420:1})

# Agrega reacciones al modelo
cobra_model.add_reaction(E1)
cobra_model.add_reaction(E2)
cobra_model.add_reaction(R1)
cobra_model.add_reaction(R2)
cobra_model.add_reaction(R3)
cobra_model.add_reaction(E3)
cobra_model.add_reaction(E4)
cobra_model.add_reaction(E5)

# Define la función objetivo
cobra_model.objective = 'E4'

In [40]:
# Iterate through the the objects in the model
print("Reactions")
print("---------")
for x in cobra_model.reactions: # tiene 8 reacciones
    print(x.id, x.name, x.reaction)
print("\nMetabolites")
print("-----------")
for x in cobra_model.metabolites: # tiene 6 metabolites
    print(x.id, x.formula)

Reactions
---------
E1 Exchange reaction of CO2  --> CO2
E2 Exchange reaction of H2  --> 2.0 H2
R1 First reaction CO2 + F420 + 2 H2 --> I1
R2 Second reaction 2 H2 + I1 --> CH4 + F420
R3 Third reaction CH4 --> F420 + 2 H2O + I1
E3 Exchange reaction of F420 CH4 --> 
E4 Exchange reaction of CH4 2 F420 --> 
E5 Exchange reaction of H2O 2 H2O --> 

Metabolites
-----------
CO2 CO2
H2 H2
F420 None
I1 None
CH4 CH4
H2O H2O


## Ejercicio

Completa el modelo metabólico

In [45]:
solution = cobra_model.optimize()
print("Valor función objetivo: ", solution.objective_value )
for i in cobra_model.reactions: # tiene 8 reacciones
    print( i.id, i.x )

Valor función objetivo:  10.0
E1 0.0
E2 10.0
R1 0.0
R2 10.0
R3 10.0
E3 0.0
E4 10.0
E5 10.0


# Análisis de soluciones alternativas de FBA


FBA no siempre da una solución única porque multiples configuraciones de flujos pueden alcanzar el mismo valor de la función objetivo. 

Incluso una red simple (la de abajo es la más simple que puedo imaginar) adolece de suficientes ecuaciones para determinar la configuración de flujos metabólicos.

<img src="https://raw.githubusercontent.com/mrivas/ModelamientoDeProcesosBiotecnologicos/master/clase5_solucionesAlternativas.png" width="500">

## Flux variability analysis

FVA (flux variability analysis) encuentra el rango de cada flujo metabólico en el valor óptimo.

In [6]:
from cobra.flux_analysis import flux_variability_analysis
flux_variability_analysis(cobra_model, cobra_model.reactions[:8])

,minimum,maximum
E1,10.0,10.0
E2,20.0,1000.0
R1,10.0,10.0
R2,10.0,990.0
R3,0.0,980.0
E3,10.0,10.0
E4,0.0,980.0
E5,0.0,980.0


Flux sampling
-------

Otra alternativa para explorar el espacio de soluciones alternativas de FBA es tomar muestrasa aleatorias. Esto se llama Flux sampling.

In [7]:
from cobra.test import create_test_model
from cobra.sampling import sample
s = sample(cobra_model, 100) # tomaremos 100 muestras aleatorias
s.head(6)

,E1,E2,R1,R2,R3,E3,E4,E5
0,5.491459,238.316857,5.491459,232.825398,227.333939,5.491459,227.333939,227.333939
1,1.291402,907.521142,1.291402,906.229741,904.938339,1.291402,904.938339,904.938339
2,9.631295,417.632144,9.631295,408.000849,398.369554,9.631295,398.369554,398.369554
3,0.225973,487.296352,0.225973,487.070380,486.844407,0.225973,486.844407,486.844407
4,5.874790,874.888987,5.874790,869.014196,863.139406,5.874790,863.139406,863.139406
5,7.404036,708.990939,7.404036,701.586903,694.182867,7.404036,694.182867,694.182867


In [8]:
#Si quiero ver solo los valores de la reacción E1
s["E1"]

0     5.491459
1     1.291402
2     9.631295
3     0.225973
4     5.874790
        ...   
95    2.123267
96    5.902578
97    0.820557
98    7.380974
99    4.708547
Name: E1, Length: 100, dtype: float64

In [9]:
#Si quiero ver el valor promedio de la reacción E1 a lo largo de las 100 muestras
s["E1"].mean()

5.277903950191351

Ejercicios
==========

1. Computa FBA de la red de *Escherichia coli*
2. Realiza FVA para la solución anterior.
3. Computa 100 muestras aleatorias del espacio de soluciones alternativas de 1. Y luego calcula el valor promedio de la reacción PYK.